In [1]:
import cmudict
import nltk
phone_dict = nltk.corpus.cmudict.dict()


In [6]:
phone_dict['joke']

[['JH', 'OW1', 'K']]

# Pun generator

## Extracting and comparing phones

In [2]:
# extracting phones from words and sentences

# consider using metaphonedoble instead of this library

def word_to_phoneme(word):
    return phone_dict[word][0]

def sentence_to_word_of_phoneme(sentence):
    """takes string sentence and returns
    list of lists of composing phones"""
    return [word_to_phoneme(word) for 
            word in sentence.lower().split()]

def subfinder_bool(mylist, pattern):
    """if a subpattern is in a list return 
    a bool"""
    matches = []
    for i in range(len(mylist)):
        if mylist[i] == pattern[0] and mylist[i:i+len(pattern)] == pattern:
            matches.append(pattern)
            return True
    return False

In [3]:
# phone comparisions

def edit_distance(w1, w2):
    """Code taken from 
    https://github.com/maxwell-schwartz/PUNchlineGenerator
    
    Levenshtein distance
    """
    
    cost = []
    
    # These may be useful for later work:
    #vowels = ['A', 'E', 'I', 'O', 'U']
    #voiced = ['B', 'D', 'G', 'J', 'L', 'M', 'N', 'R', 'V', 'W', 'Y', 'Z']
    #unvoiced = ['C', 'F', 'H', 'K', 'P', 'S', 'T']
    
    for i in range(len(w1)+1):
        x = []
        for j in range(len(w2)+1):
            x.append(0)
        cost.append(x)
    
    for i in range(len(w1)+1):
        cost[i][0] = i
    for j in range(len(w2)+1):
        cost[0][j] = j
        
    # baseline costs
    del_cost = 2
    add_cost = 2
    sub_cost = 1
    
    for i in range(1, len(w1)+1):
        for j in range(1, len(w2)+1):
            if w1[i-1] == w2[j-1]:
                sub_cost = 0
            else:
                sub_cost = 2
            # get the totals
            del_total = cost[i-1][j] + del_cost
            add_total = cost[i][j-1] + add_cost
            sub_total = cost[i-1][j-1] + sub_cost
            # choose the lowest cost from the options
            options = [del_total, add_total, sub_total]
            options.sort()
            cost[i][j] = options[0]

    return cost[-1][-1]

In [4]:
def debug_distance(word1, word2):
    print(phonetic_distance(word1, word2))
    print(word_to_phones(word1))
    print(word_to_phones(word2))

In [5]:
def phonetic_distance(word1, word2):
    """compares two words and returns phonetic
    distance"""
    phoneme1 = word_to_phoneme(word1.lower())
    phoneme2 = word_to_phoneme(word2.lower())
    return edit_distance(phoneme1, phoneme2)

In [13]:
def enumerate_PD_pun_subs(sentence, possible_words, max_distance=5, max_return=10):
    """
    Takes a sentence and possible words and creates returns an array of possible 
    pun substituions based on phonetic distance
    """
    output = []
    sentence_words = list(sentence.split())
    for word_index, word in enumerate(sentence_words):
        for pos_word in possible_words:
            if pos_word in word:
                # This substituion would be meaningless
                continue
                
            dist = phonetic_distance(word, pos_word)
            if dist <= max_distance:
                # Decrease the distance 
                output.append((pos_word, word_index, dist))
    output.sort(key=lambda tup: tup[2])
    return output

In [14]:
def substitute_pun(sentence, sub_tuple):
    """Takes a sentence 
    and a touple of (word, index, and score)
    and makes a sentence
    """
    sentence_words = list(sentence.split())
    sentence_words[sub_tuple[1]] = sub_tuple[0]
    return ' '.join(word for word in sentence_words)


In [15]:
sentence = 'There was a man who wanted to make a pun in a pinch'
possible_sub_words = ['music', 'peel', 'thyme', 
                         'mime', 'inside', 'remind', 
                         'mess', 'nest', 'credential', 'special', 
                        'kiss', 'banter', 'flatter']


for output in enumerate_PD_pun_subs(sentence, possible_sub_words, max_distance=4):
    print(output)
    print(substitute_pun(sentence, output))

('mess', 0, 4)
mess was a man who wanted to make a pun in a pinch
('peel', 2, 4)
There was peel man who wanted to make a pun in a pinch
('thyme', 2, 4)
There was thyme man who wanted to make a pun in a pinch
('mime', 2, 4)
There was mime man who wanted to make a pun in a pinch
('mess', 2, 4)
There was mess man who wanted to make a pun in a pinch
('kiss', 2, 4)
There was kiss man who wanted to make a pun in a pinch
('mime', 3, 4)
There was a mime who wanted to make a pun in a pinch
('mess', 3, 4)
There was a mess who wanted to make a pun in a pinch
('mime', 7, 4)
There was a man who wanted to mime a pun in a pinch
('mess', 7, 4)
There was a man who wanted to mess a pun in a pinch
('peel', 8, 4)
There was a man who wanted to make peel pun in a pinch
('thyme', 8, 4)
There was a man who wanted to make thyme pun in a pinch
('mime', 8, 4)
There was a man who wanted to make mime pun in a pinch
('mess', 8, 4)
There was a man who wanted to make mess pun in a pinch
('kiss', 8, 4)
There was a man

In [21]:
def insert_pun(sentence, possible_words, max_distance=5, max_return=10):
    """
    function to generate a num
    """
#     best_distance = max_distance
#     best_index = None
#     best_word = None
    sentence_words = list(sentence.split())
    for word_index, word in enumerate(sentence_words):
        for pos_word in possible_words:
            if pos_word in word:
                # This substituion would be meaningless
                continue
            dist = phonetic_distance(word, pos_word)
#             
            if dist <= best_distance:
                # Decrease the distance 
                best_distance += -1
                best_index = word_index
                best_word = pos_word

    if best_word is None:
        return 'no substitution found \n' + sentence
    
    sentence_words[best_index] = best_word
    
    return ' '.join(word for word in sentence_words)
        